In [1]:
import os
print(os.getcwd())

d:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction


## Alternative method using donwsampled PNG by PNG for feature extraction

## Pipeline for feature extraction using UNI_2h

### Uni2-h feature extrator fivecrop

In [2]:
import torch
from PIL import Image
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from torchvision import transforms

class UNI2ExtractorFiveCrop():
    def __init__(self, device='cuda'):
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')

        timm_kwargs = {
            'img_size': 224,
            'patch_size': 14,
            'depth': 24,
            'num_heads': 24,
            'init_values': 1e-5,
            'embed_dim': 1536,
            'mlp_ratio': 2.66667 * 2,
            'num_classes': 0,
            'no_embed_class': True,
            'mlp_layer': timm.layers.SwiGLUPacked,
            'act_layer': torch.nn.SiLU,
            'reg_tokens': 8,
            'dynamic_img_size': True
        }

        self.model = timm.create_model("hf-hub:MahmoodLab/UNI2-h", pretrained=True, **timm_kwargs).to(self.device)
        self.model.eval()

        # Get base transform for individual crops
        config = resolve_data_config(self.model.pretrained_cfg, model=self.model)
        self.eval_transform = create_transform(**config)

        # FiveCrop transform: crop → transform each crop → stack into 5x3x224x224
        self.fivecrop_transform = transforms.Compose([
            transforms.FiveCrop(256),  # five 256x256 crops from the 512x512 image
            transforms.Lambda(lambda crops: torch.stack([self.eval_transform(c) for c in crops]))
        ])

    def extract_fivecrop_features(self, image: Image.Image | str) -> torch.Tensor:
        """
        Extract 5xD features from one image using FiveCrop.
        Returns: Tensor of shape (5, D)
        """
        if isinstance(image, str):
            image = Image.open(image).convert("RGB")
        else:
            image = image.convert("RGB")

        crops_tensor = self.fivecrop_transform(image).to(self.device)  # shape: (5, 3, 224, 224)

        with torch.no_grad():
            features = self.model(crops_tensor)  # shape: (5, D)

        return features.cpu()

### Old Final Pipeline (Incorrect)

In [ ]:
from huggingface_hub import login
from transformers import AutoModel
from PIL import Image
import torch
import torch.nn.functional as F
from torchvision import transforms
import os
import glob
from tqdm import tqdm
import logging
import gc
import time
from concurrent.futures import ThreadPoolExecutor
import threading
from queue import Queue
import multiprocessing as mp

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Global model instance to avoid reloading
_model_instance = None
_model_lock = threading.Lock()

def get_model_instance():
    global _model_instance
    if _model_instance is None:
        with _model_lock:
            if _model_instance is None:
                logger.info("Loading UNI2 model...")
                _model_instance = UNI2ExtractorFiveCrop()
    return _model_instance

class OptimizedBatchProcessor:
    def __init__(self, batch_size=64, num_workers=4):
        self.batch_size = batch_size
        self.num_workers = num_workers
        
        self.extractor = get_model_instance()
        self.transform = self.extractor.fivecrop_transform
        self.device = self.extractor.device

    def load_images_parallel(self, image_paths):
        def load_single(path):
            try:
                if not os.path.exists(path):
                    return None, path
                with Image.open(path) as img:
                    img_rgb = img.convert("RGB")
                    crops = self.transform(img_rgb)
                    return crops, path
            except Exception as e:
                logger.warning(f"Failed to load {path}: {e}")
                return None, path

        with ThreadPoolExecutor(max_workers=self.num_workers) as executor:
            results = list(executor.map(load_single, image_paths))
        
        valid_crops = []
        valid_paths = []
        for crops, path in results:
            if crops is not None:
                valid_crops.append(crops)
                valid_paths.append(path)
        
        return valid_crops, valid_paths

    def process_mega_batch(self, image_paths):
        if not image_paths:
            return []

        valid_crops, valid_paths = self.load_images_parallel(image_paths)
        if not valid_crops:
            return []

        results = []
        try:
            batch_tensor = torch.stack(valid_crops)  # (N, 5, C, H, W)
            N, crops_per_img, C, H, W = batch_tensor.shape
            batch_tensor = batch_tensor.view(N * crops_per_img, C, H, W).to(self.device)

            with torch.no_grad():
                features = self.extractor.model(batch_tensor)

            features = features.view(N, crops_per_img, -1).cpu()

            for i, path in enumerate(valid_paths):
                print(features.shape)
                results.append((features[i], path))

        except RuntimeError as e:
            if "out of memory" in str(e):
                torch.cuda.empty_cache()
                mid = len(image_paths) // 2
                results.extend(self.process_mega_batch(image_paths[:mid]))
                results.extend(self.process_mega_batch(image_paths[mid:]))
            else:
                logger.error(f"Error: {e}")
                torch.cuda.empty_cache()

        return results


def save_features_parallel(results, patch_root, save_root, num_workers=4):
    """Save features in parallel."""
    def save_single(item):
        features, patch_path = item
        try:
            # Create save path
            relative_path = os.path.relpath(patch_path, patch_root)
            slide_folder, patch_filename = os.path.split(relative_path)
            patch_base = os.path.splitext(patch_filename)[0]
            
            save_dir = os.path.join(save_root, slide_folder)
            os.makedirs(save_dir, exist_ok=True)
            
            save_path = os.path.join(save_dir, patch_base + ".pt")
            torch.save(features, save_path)
            return True
        except Exception as e:
            logger.error(f"Error saving {patch_path}: {e}")
            return False
    
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        success_results = list(executor.map(save_single, results))
    
    return sum(success_results)

def process_patches_ultra_fast(
    patch_data_dir: str = r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\test_data",
    output_root: str = r"D:\Aamir Gulzar\KSA_project2\dataset\Features_test\Conch_v1_5_fivecrop",
    slide_meta_csv: str = None,
    non_white_csv: str = None
):
    """Ultra-fast processing with all optimizations and validation checks."""
    import pandas as pd
    
    if not os.path.exists(patch_data_dir):
        logger.error(f"Patch root directory not found: {patch_data_dir}")
        return
    
    # Load validation data if provided
    valid_patch_names = None
    slide_meta_df = None
    
    if non_white_csv and os.path.exists(non_white_csv):
        logger.info("Loading non-white patch validation...")
        non_white_df = pd.read_csv(non_white_csv)
        valid_patch_names = set(non_white_df['patch_name'].astype(str))
        logger.info(f"Found {len(valid_patch_names)} valid non-white patches")
    
    if slide_meta_csv and os.path.exists(slide_meta_csv):
        logger.info("Loading slide metadata...")
        slide_meta_df = pd.read_csv(slide_meta_csv)
        logger.info(f"Loaded metadata for {len(slide_meta_df)} slides")
    
    # Initialize processor
    processor = OptimizedBatchProcessor(batch_size=128, num_workers=8)
    os.makedirs(output_root, exist_ok=True)
    
    # Get all slide directories
    slide_dirs = glob.glob(os.path.join(patch_data_dir, "*"))
    logger.info(f"Found {len(slide_dirs)} slide directories")
    
    total_processed = 0
    total_skipped = 0
    total_failed = 0
    start_time = time.time()
    
    for slide_dir in tqdm(slide_dirs, desc="Processing slides"):
        slide_name = os.path.basename(slide_dir)
        slide_output_dir = os.path.join(output_root, slide_name)

        if os.path.exists(slide_output_dir) and len(os.listdir(slide_output_dir)) > 0:
            logger.info(f"\n📄 Skipping slide {slide_name}: Output directory already exists and is not empty.")
            total_skipped += 1
            continue

        logger.info(f"\n📄 Processing slide: {slide_name}")
        
        # Validate slide metadata if provided
        if slide_meta_df is not None:
            meta_row = slide_meta_df[slide_meta_df["renamed"].str.lower() == slide_name.lower()]
            if meta_row.empty:
                logger.warning(f"  ❌ Skipped: Slide metadata not found for {slide_name}")
                total_skipped += 1
                continue
            
            try:
                obj_power = int(meta_row.iloc[0]["Objective Power"])
                patch_size_level0 = 1024 if obj_power == 40 else 512 if obj_power == 20 else None
                if patch_size_level0 is None:
                    logger.warning(f"  ❌ Unsupported objective power: {obj_power}")
                    total_skipped += 1
                    continue
                logger.info(f"  ✅ Objective power: {obj_power}x, patch size: {patch_size_level0}")
            except Exception as e:
                logger.error(f"  ❌ Error extracting objective power: {e}")
                total_failed += 1
                continue
        
        # Get all patches for this slide
        all_patches = glob.glob(os.path.join(slide_dir, "*.png"))
        
        # Filter by valid patch names if provided
        if valid_patch_names:
            valid_patches = [p for p in all_patches if os.path.basename(p) in valid_patch_names]
            logger.info(f"  📊 Filtered patches: {len(valid_patches)}/{len(all_patches)} valid non-white patches")
        else:
            valid_patches = all_patches
            logger.info(f"  📊 Found {len(valid_patches)} patches (no filtering applied)")
        
        if not valid_patches:
            logger.warning(f"  ❌ No valid patches found for {slide_name}")
            total_skipped += 1
            continue
        
        # Check which patches need processing
        os.makedirs(slide_output_dir, exist_ok=True)
        
        remaining_patches = []
        for patch_path in valid_patches:
            patch_name = os.path.splitext(os.path.basename(patch_path))[0]
            save_path = os.path.join(slide_output_dir, patch_name + ".pt")
            if not os.path.exists(save_path):
                remaining_patches.append(patch_path)
        
        if not remaining_patches:
            logger.info(f"  ✅ All patches already processed for {slide_name}")
            continue
        
        logger.info(f"  🔄 Processing {len(remaining_patches)} new patches (skipping {len(valid_patches) - len(remaining_patches)} existing)")
        
        # Process patches in mega batches for this slide
        slide_processed = 0
        mega_batch_size = 1  # Smaller batches per slide for better progress tracking
        
        for i in range(0, len(remaining_patches), mega_batch_size):
            batch_patches = remaining_patches[i:i + mega_batch_size]
            
            try:
                # Process mega batch
                results = processor.process_mega_batch(batch_patches)
                
                # Save results for this batch
                if results:
                    saved_count = save_features_parallel(results, patch_data_dir, output_root)
                    slide_processed += saved_count
                    total_processed += saved_count
                
            except Exception as e:
                logger.error(f"  ❌ Error processing batch for {slide_name}: {e}")
                total_failed += len(batch_patches)
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
        
        logger.info(f"  ✅ Completed {slide_name}: {slide_processed} patches processed")
        
        # Memory cleanup after each slide
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    # Final statistics
    total_time = time.time() - start_time
    logger.info(f"\n🚀 ULTRA-FAST PIPELINE COMPLETE! 🚀")
    logger.info(f"📊 FINAL STATS:")
    logger.info(f"   ✅ Processed: {total_processed} patches")
    logger.info(f"   ⏭️  Skipped: {total_skipped} slides")
    logger.info(f"   ❌ Failed: {total_failed} patches")
    logger.info(f"   ⏱️  Total time: {total_time/60:.2f} minutes")
    logger.info(f"   🚀 Average rate: {total_processed/total_time:.1f} patches/second")
    
    return total_processed, total_skipped, total_failed
    

def benchmark_test():
    """Quick benchmark test."""
    patch_root = r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\test_data"
    test_paths = glob.glob(os.path.join(patch_root, "*", "*.png"))[:50]  # Test with 50 images
    
    if len(test_paths) < 10:
        logger.error("Need at least 10 test images")
        return False
    
    logger.info(f"🧪 Benchmark test with {len(test_paths)} images...")
    
    try:
        processor = OptimizedBatchProcessor(batch_size=128, num_workers=8)
        
        start = time.time()
        results = processor.process_mega_batch(test_paths)
        elapsed = time.time() - start
        
        rate = len(results) / elapsed
        logger.info(f"✅ Benchmark successful!")
        logger.info(f"   Rate: {rate:.1f} images/second")
        logger.info(f"   Estimated total time: {6695/rate/60:.1f} minutes")
        
        return rate > 2.0  # Must be at least 2 imgs/sec
        
    except Exception as e:
        logger.error(f"❌ Benchmark failed: {e}")
        return False

if __name__ == "__main__":
    # Configuration - Update these paths as needed
    PATCH_DATA_DIR = r"D:\Aamir Gulzar\KSA_project2\dataset\patch_data"
    OUTPUT_ROOT = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Uni_2h_fivecrop"
    
    # Optional validation files - set to None to skip validation
    SLIDE_META_CSV = r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\slide_metadata_filtered.csv"  
    NON_WHITE_CSV = r"D:\Aamir Gulzar\KSA_project2\dataset\final_merged_without_white.csv" 
    
    # Run benchmark first
    if benchmark_test():
        print("\n" + "="*60)
        print("🚀 BENCHMARK PASSED - READY FOR ULTRA-FAST PROCESSING! 🚀")
        print("="*60)
        
        # Show configuration
        print(f"📁 Input directory: {PATCH_DATA_DIR}")
        print(f"💾 Output directory: {OUTPUT_ROOT}")
        print(f"📊 Slide metadata: {'✅ Enabled' if SLIDE_META_CSV else '❌ Disabled'}")
        print(f"🎨 Non-white filter: {'✅ Enabled' if NON_WHITE_CSV else '❌ Disabled'}")
        
        response = input("\nProceed with processing? (y/n): ")
        if response.lower() == 'y':
            try:
                processed, skipped, failed = process_patches_ultra_fast(
                    patch_data_dir=PATCH_DATA_DIR,
                    output_root=OUTPUT_ROOT,
                    slide_meta_csv=SLIDE_META_CSV,
                    non_white_csv=NON_WHITE_CSV
                )
                
                print("\n" + "="*60)
                print("🎉 PROCESSING COMPLETE! 🎉")
                print(f"✅ Successfully processed: {processed} patches")
                if skipped > 0:
                    print(f"⏭️  Skipped slides: {skipped}")
                if failed > 0:
                    print(f"❌ Failed patches: {failed}")
                print("="*60)
                
            except KeyboardInterrupt:
                logger.info("\n⏹️  Processing interrupted by user")
            except Exception as e:
                logger.error(f"\n💥 Processing failed: {e}")
        else:
            logger.info("Processing cancelled by user.")
    else:
        logger.error("❌ Benchmark failed. Please check your setup.")

INFO:__main__:🧪 Benchmark test with 50 images...
INFO:__main__:Loading UNI2 model...
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (MahmoodLab/UNI2-h)
INFO:__main__:✅ Benchmark successful!
INFO:__main__:   Rate: 4.5 images/second
INFO:__main__:   Estimated total time: 24.8 minutes



🚀 BENCHMARK PASSED - READY FOR ULTRA-FAST PROCESSING! 🚀
📁 Input directory: D:\Aamir Gulzar\KSA_project2\dataset\patch_data
💾 Output directory: D:\Aamir Gulzar\KSA_project2\dataset\Features\Uni_2h_fivecrop
📊 Slide metadata: ✅ Enabled
🎨 Non-white filter: ✅ Enabled


INFO:__main__:Loading non-white patch validation...
INFO:__main__:Found 1377933 valid non-white patches
INFO:__main__:Loading slide metadata...
INFO:__main__:Loaded metadata for 425 slides
INFO:__main__:Found 417 slide directories
Processing slides:   0%|          | 0/417 [00:00<?, ?it/s]INFO:__main__:
📄 Skipping slide TCGA-3L-AA1B_nonMSIH: Output directory already exists and is not empty.
Processing slides:   0%|          | 1/417 [00:00<01:47,  3.88it/s]INFO:__main__:
📄 Skipping slide TCGA-4N-A93T_nonMSIH: Output directory already exists and is not empty.
Processing slides:   0%|          | 2/417 [00:00<02:16,  3.05it/s]INFO:__main__:
📄 Skipping slide TCGA-5M-AAT4_nonMSIH: Output directory already exists and is not empty.
Processing slides:   1%|          | 3/417 [00:01<02:57,  2.34it/s]INFO:__main__:
📄 Skipping slide TCGA-5M-AAT6_MSIH: Output directory already exists and is not empty.
Processing slides:   1%|          | 4/417 [00:03<06:47,  1.01it/s]INFO:__main__:
📄 Skipping slide TC

### Testing

In [3]:
# FiveCrop feature extraction function
import glob
import os
from tqdm import tqdm
import torch

# Set paths
patch_root = r"D:\Aamir Gulzar\KSA_project2\dataset\patch_data"
save_root = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Uni_2h_fivecrop_Updated"

# Initialize extractor
extractor = UNI2ExtractorFiveCrop()

# Loop through all PNG patches
all_patch_paths = glob.glob(os.path.join(patch_root, "*", "*.png"))

for patch_path in tqdm(all_patch_paths):
    try:
        features = extractor.extract_fivecrop_features(patch_path)  # shape: (5, D)

        # Extract subfolder and filename
        relative_path = os.path.relpath(patch_path, patch_root)
        slide_folder, patch_filename = os.path.split(relative_path)
        patch_base = os.path.splitext(patch_filename)[0]  # remove .png

        # Create save path
        save_dir = os.path.join(save_root, slide_folder)
        os.makedirs(save_dir, exist_ok=True)

        save_path = os.path.join(save_dir, patch_base + ".pt")
        torch.save(features, save_path)
    
    except Exception as e:
        print(f"Error processing {patch_path}: {e}")

  0%|          | 895/1460238 [02:12<59:55:38,  6.76it/s]


KeyboardInterrupt: 

## Final Pipeline

In [3]:
import os
import glob
import pandas as pd
import torch
from tqdm import tqdm

def run_full_pipeline_uni2_fivecrop(
    patch_data_dir: str,
    extractor,  # instance of UNI2ExtractorFiveCrop
    non_white_csv: str,
    output_root: str
):
    # Load non-white metadata
    non_white_df = pd.read_csv(non_white_csv)
    valid_patch_names = set(non_white_df['patch_name'].astype(str))

    # Loop over slide directories
    slide_dirs = glob.glob(os.path.join(patch_data_dir, "*"))
    for slide_dir in tqdm(slide_dirs, desc="🧩 Processing slides", unit="slide", position=0):
        slide_name = os.path.basename(slide_dir)
        patch_feat_dir = os.path.join(output_root, slide_name)
        os.makedirs(patch_feat_dir, exist_ok=True)

        # Filter valid non-white patches
        all_patches = glob.glob(os.path.join(slide_dir, "*.png"))
        valid_patch_paths = [p for p in all_patches if os.path.basename(p) in valid_patch_names]

        if not valid_patch_paths:
            tqdm.write(f"  ❌ No valid non-white patches found for {slide_name}")
            continue

        # Skip already processed patches
        existing_feats = set(os.path.splitext(f.name)[0] for f in os.scandir(patch_feat_dir) if f.name.endswith(".pt"))
        to_process = [p for p in valid_patch_paths if os.path.splitext(os.path.basename(p))[0] not in existing_feats]

        if not to_process:
            tqdm.write(f"  ⏭️ Skipped: All {len(valid_patch_paths)} patches already processed for {slide_name}")
            continue

        # Patch-level tqdm bar
        for patch_path in tqdm(to_process, desc=f"🔄 {slide_name}", unit="patch", position=1, leave=False):
            patch_name = os.path.splitext(os.path.basename(patch_path))[0]
            save_path = os.path.join(patch_feat_dir, patch_name + ".pt")

            try:
                features = extractor.extract_fivecrop_features(patch_path)  # shape (5, D)
                torch.save(features, save_path)
            except Exception as e:
                tqdm.write(f"  ❌ Failed to process patch {patch_path}: {e}")
                continue

        tqdm.write(f"  ✅ Done: Processed {len(to_process)} new patches (total: {len(valid_patch_paths)}) for {slide_name}")

In [5]:
extractor = UNI2ExtractorFiveCrop()

run_full_pipeline_uni2_fivecrop(
    patch_data_dir=r"D:\Aamir Gulzar\KSA_project2\dataset\patch_data",
    extractor=extractor,
    non_white_csv=r"D:\Aamir Gulzar\KSA_project2\dataset\final_merged_without_white.csv",
    output_root=r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Uni_2h_fivecrop_Updated"
)

🧩 Processing slides:   0%|          | 1/417 [00:00<00:58,  7.07slide/s]

  ⏭️ Skipped: All 1878 patches already processed for TCGA-3L-AA1B_nonMSIH


🧩 Processing slides:   0%|          | 2/417 [00:00<01:21,  5.11slide/s]

  ⏭️ Skipped: All 4240 patches already processed for TCGA-4N-A93T_nonMSIH


🧩 Processing slides:   1%|          | 3/417 [00:00<01:39,  4.16slide/s]

...

  ✅ Done: Processed 5746 new patches (total: 5746) for TCGA-SS-A7HO_nonMSIH


🧩 Processing slides: 100%|█████████▉| 416/417 [8:24:30<13:55, 835.42s/slide]

  ✅ Done: Processed 5462 new patches (total: 5462) for TCGA-T9-A92H_nonMSIH


🧩 Processing slides: 100%|██████████| 417/417 [8:36:33<00:00, 74.32s/slide] 


  ✅ Done: Processed 4532 new patches (total: 4532) for TCGA-WS-AB45_MSIH


## Check .pt file

In [5]:
import torch
f = torch.load(r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Uni_2h_fivecrop_Updated\TCGA-3L-AA1B_nonMSIH\TCGA-3L-AA1B_nonMSIH_x10288_y12512_patch00159.pt")
print(f.shape)       # Should be torch.Size([5, 1536])
print(f.element_size(), f.numel(), f.element_size() * f.numel() / 1024, "KB")

torch.Size([5, 1536])
4 7680 30.0 KB


In [7]:
import torch

def inspect_pt_file(pt_path):
    """Inspect the contents of a PyTorch .pt file"""
    print(f"Inspecting PT file: {pt_path}")
    print("=" * 50)

    data = torch.load(pt_path, map_location='cpu')

    if isinstance(data, dict):
        for key, value in data.items():
            print(f"Key: {key}")
            if isinstance(value, torch.Tensor):
                print(f"  Type: Tensor")
                print(f"  Shape: {tuple(value.shape)}")
                print(f"  Dtype: {value.dtype}")
                print("  First 10 values in each of the 5 rows:")
                for i in range(min(5, value.shape[0])):  # Only first 5 rows
                    row_values = value[i][:10].tolist()
                    print(f"    Row {i+1}: {row_values}")
            else:
                print(f"  Type: {type(value)}")
                print(f"  Value: {value}")
            print()
    elif isinstance(data, torch.Tensor):
        print(f"Single Tensor:")
        print(f"  Shape: {tuple(data.shape)}")
        print(f"  Dtype: {data.dtype}")
        print("  First 10 values in each of the 5 rows:")
        for i in range(min(5, data.shape[0])):
            row_values = data[i][:10].tolist()
            print(f"    Row {i+1}: {row_values}")
    else:
        print("The .pt file is not a dictionary or tensor. Type:", type(data))
        print("Value preview:", str(data)[:1500])  # Print partial value

# Example usage
pt_path = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Uni_2h_fivecrop_Updated\TCGA-3L-AA1B_nonMSIH\TCGA-3L-AA1B_nonMSIH_x10288_y12512_patch00159.pt"
inspect_pt_file(pt_path)


Inspecting PT file: D:\Aamir Gulzar\KSA_project2\dataset\Features\Uni_2h_fivecrop_Updated\TCGA-3L-AA1B_nonMSIH\TCGA-3L-AA1B_nonMSIH_x10288_y12512_patch00159.pt
Single Tensor:
  Shape: (5, 1536)
  Dtype: torch.float32
  First 10 values in each of the 5 rows:
    Row 1: [-0.04799284413456917, -0.25558799505233765, 0.26900842785835266, -0.5940723419189453, -0.6035252213478088, 0.16037911176681519, 0.3446435332298279, 0.3134009838104248, -0.13428805768489838, -0.3708030581474304]
    Row 2: [-0.06264611333608627, 0.14632725715637207, 0.13261474668979645, -0.46976330876350403, -0.5787501335144043, 0.6793582439422607, 0.3106735050678253, 0.13066841661930084, -0.413820743560791, -0.31830263137817383]
    Row 3: [0.20704224705696106, 0.0770830437541008, -0.3512730598449707, -0.5216561555862427, -0.607268214225769, 0.384053111076355, 0.4551374912261963, 0.3418516516685486, 0.012581361457705498, -0.3256070911884308]
    Row 4: [-0.03736783564090729, 0.3507113456726074, 0.06264858692884445, -0.49

In [8]:
import torch
import os

def run_sanity_checks(tensor: torch.Tensor):
    print("Running sanity checks...\n")

    # 1. Shape
    if tensor.shape != (5, 1536):
        print(f"[❌] Shape mismatch: Expected (5, 1536), got {tuple(tensor.shape)}")
    else:
        print(f"[✅] Shape OK: {tuple(tensor.shape)}")

    # 2. Dtype
    if tensor.dtype not in [torch.float32, torch.float16]:
        print(f"[❌] Unexpected dtype: {tensor.dtype}")
    else:
        print(f"[✅] Dtype OK: {tensor.dtype}")

    # 3. NaN or Inf
    if not torch.isfinite(tensor).all():
        nan_count = torch.isnan(tensor).sum().item()
        inf_count = torch.isinf(tensor).sum().item()
        print(f"[❌] Invalid values found — NaNs: {nan_count}, Infs: {inf_count}")
    else:
        print(f"[✅] No NaN or Inf values")

    # 4. Value range
    min_val = tensor.min().item()
    max_val = tensor.max().item()
    print(f"[ℹ️] Value range: min={min_val:.4f}, max={max_val:.4f}")
    if abs(min_val) > 1e6 or abs(max_val) > 1e6:
        print("[⚠️] Warning: Value range is unusually large.")

    # 5. L2 norm (per row)
    norms = tensor.norm(p=2, dim=1)
    print(f"[ℹ️] L2 Norms (per row): {norms.tolist()}")
    if (norms == 0).any():
        print("[❌] Zero vector detected (row with all zeros)")
    else:
        print("[✅] All rows have non-zero magnitude")

    # 6. Row consistency (std across rows)
    row_std = torch.std(tensor, dim=1)
    print(f"[ℹ️] Row-wise std dev: {row_std.tolist()}")
    if (row_std == 0).any():
        print("[⚠️] Warning: At least one row is constant (no variance)")

    print("\nSanity check completed.")

# Example usage
pt_path = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Uni_2h_fivecrop_Updated\TCGA-3L-AA1B_nonMSIH\TCGA-3L-AA1B_nonMSIH_x10288_y12512_patch00159.pt"
data = torch.load(pt_path, map_location='cpu')

# If it's a dict with a tensor inside
if isinstance(data, dict):
    for key in data:
        if isinstance(data[key], torch.Tensor) and data[key].shape == (5, 1536):
            run_sanity_checks(data[key])
            break
else:
    run_sanity_checks(data)


Running sanity checks...

[✅] Shape OK: (5, 1536)
[✅] Dtype OK: torch.float32
[✅] No NaN or Inf values
[ℹ️] Value range: min=-1.1811, max=1.1903
[ℹ️] L2 Norms (per row): [11.946415901184082, 13.388145446777344, 13.688057899475098, 14.105144500732422, 12.71574592590332]
[✅] All rows have non-zero magnitude
[ℹ️] Row-wise std dev: [0.3049180805683136, 0.34171488881111145, 0.3493548035621643, 0.3600170314311981, 0.324552446603775]

Sanity check completed.
